In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
class myCallback (tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.98 and logs.get('val_accuracy')>0.95):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

In [8]:
def create_model():
    mobilenet_v2 = "https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/small-100-224-feature-vector/versions/1"
    mobile_net_layers = hub.KerasLayer(mobilenet_v2, input_shape=(224,224,3))
    mobile_net_layers.trainable = False

    model = tf.keras.Sequential([
        mobile_net_layers,
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(22,activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

    train_dir = '../data/training/'
    validation_dir = '../data/validation/'

    train_datagen = ImageDataGenerator(
        rescale=1. / 255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        rescale=1. / 255.
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size=5,
        class_mode='categorical',
        target_size=(224, 224)
    )

    callback = myCallback()

    model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        callbacks=callback
    )
    
    return model

In [5]:
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 22)                22550     
                                                                 
Total params: 1552518 (5.92 MB)
Trainable params: 22550 (88.09 KB)
Non-trainable params: 1529968 (5.84 MB)
_________________________________________________________________
Found 1480 images belonging to 22 classes.
Found 213 images belonging to 22 classes.
Epoch 1/100



296/296 [==============================] - 27s 80ms/step - loss: 1.4819 - accuracy: 0.6115 - val_loss: 0.9895 - val_accuracy: 0.7277
Epoch 2/100
296/296 [==============================] - 16s 54ms/step - loss: 0.5229 - accuracy: 0.8716 - val_loss: 0.7500 - val_accuracy: 0.7934
Epoch 3/100
296/296 [==============================] - 17s 57ms/step - loss: 0.3660 - accuracy: 0.8993 - val_loss: 0.6546 - val_accuracy: 0.8075
Epoch 4/100
296/296 [==============================] - 16s 54ms/step - loss: 0.2631 - accuracy: 0.9345 - val_loss: 0.6208 - val_accuracy: 0.8075
Epoch 5/100
296/296 [==============================] - 16s 53ms/step - loss: 0.2191 - accuracy: 0.9459 - val_loss: 0.5940 - val_accuracy: 0.7934
Epoch 6/100
296/296 [==============================] - 16s 53ms/step - loss: 0.1823 - accuracy: 0.9622 - val_loss: 0.5757 - val_accuracy: 0.7934
Epoch 7/100
296/296 [==============================] - 16s 53ms/step - loss: 0.1691 - accuracy: 0.9568 - val_loss: 0.5564 - val_accuracy: 0.81

In [13]:
# Specify the local image path
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

local_image_path = '../data/validation/Pisang/Cavendish-banana--8-_jpg.rf.5e6303ef404896e78d98a909843615bf.jpg'

# Make predictions for the local image
img = image.load_img(local_image_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
train_dir = "../data/training/"
classes = [f for f in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, f))]
predictions = model.predict(img_array)
print(predictions * 100)

predicted_class = np.argmax(predictions)
predicted_food = classes[predicted_class]

print(f"The predicted class is: {predicted_class}")
print(f"The predicted food is: {predicted_food}")

1/1 [==============================] - 0s 30ms/step
[[4.43229204e-08 3.07136122e-11 1.33992661e-11 2.60535239e-11
  1.71031855e-10 1.51088256e-11 6.20145428e-08 5.48343338e-15
  4.50033699e-09 2.38586516e-17 9.29614025e-12 4.33727524e-14
  1.28019570e-13 9.99999771e+01 2.11223435e-11 1.45806527e-11
  2.70192177e-05 2.54407162e-08 5.94493628e-08 4.56032944e-13
  2.22820609e-13 1.11765476e-13]]
The predicted class is: 13
The predicted food is: Pisang
